### Import Libraries

In [1]:
import psycopg2
import pandas as pd


### Load CSV

In [2]:
df = pd.read_csv(r"C:\Users\Administrator\Downloads\Customer-Experience-Analytics\Customer-Experience-Analytics\data\processed\playstore_reviews_clean.csv")
df.head()  # Preview the first 5 rows


,review,rating,date,bank,source
0,maaliif daddafee install gaafata,3,2025-11-28,Commercial Bank of Ethiopia (CBE),Google Play
1,good app,5,2025-11-28,Commercial Bank of Ethiopia (CBE),Google Play
2,This application is very important and advanta...,5,2025-11-27,Commercial Bank of Ethiopia (CBE),Google Play
3,why didn't work this app?,1,2025-11-27,Commercial Bank of Ethiopia (CBE),Google Play
4,The app makes our life easier. Thank you CBE!,5,2025-11-27,Commercial Bank of Ethiopia (CBE),Google Play


### Connect to PostgreSQL

In [3]:
conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="postgres",
    password="postgres"
)
cursor = conn.cursor()


In [6]:
conn.rollback()


### Insert Unique Banks

In [7]:
unique_banks = df['bank'].unique()
bank_id_mapping = {}

for i, bank_name in enumerate(unique_banks, start=1):
    app_name = f"{bank_name} App"  # Placeholder app name
    cursor.execute(
        "INSERT INTO banks (bank_name, app_name) VALUES (%s, %s) ON CONFLICT (bank_name) DO NOTHING RETURNING bank_id",
        (bank_name, app_name)
    )
    result = cursor.fetchone()
    bank_id = result[0] if result else i
    bank_id_mapping[bank_name] = bank_id


### Insert Reviews

In [8]:
for _, row in df.iterrows():
    bank_id = bank_id_mapping[row["bank"]]

    cursor.execute("""
        INSERT INTO reviews 
        (bank_id, review_text, rating, review_date, source)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        bank_id,
        row["review"],
        row["rating"],
        row["date"],
        row["source"]
    ))

conn.commit()
print("All reviews inserted successfully!")


All reviews inserted successfully!


### Count total reviews

In [9]:
cursor.execute("SELECT COUNT(*) FROM reviews;")
cursor.fetchone()


(888,)

### Reviews per bank

In [10]:
cursor.execute("""
SELECT b.bank_name, COUNT(*) 
FROM reviews r 
JOIN banks b ON r.bank_id = b.bank_id
GROUP BY b.bank_name;
""")
cursor.fetchall()


[('Bank of Abyssinia (BOA)', 205),
 ('Dashen Bank', 332),
 ('Commercial Bank of Ethiopia (CBE)', 351)]

### Average rating per bank

In [11]:
cursor.execute("""
SELECT b.bank_name, ROUND(AVG(r.rating),2) 
FROM reviews r 
JOIN banks b ON r.bank_id = b.bank_id
GROUP BY b.bank_name;
""")
cursor.fetchall()


[('Bank of Abyssinia (BOA)', Decimal('3.05')),
 ('Dashen Bank', Decimal('3.89')),
 ('Commercial Bank of Ethiopia (CBE)', Decimal('3.94'))]

### Close Database Connection

In [12]:
cursor.close()
conn.close()
print("Connection closed.")


Connection closed.
